# LangChain: Models: Prompts and Output parsers

**Content:**
- Direct calls to OpenAI
- API calls through LangChain
    - Prompts
    - Models
    - Output parsers

## 1.- Get OpenAI API Key

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ["OPENAI_KEY"]


In [4]:
from openai import OpenAI

# Helper function for completing text given a specific model from the OpenAI API
def get_completion(prompt, model="gpt-4o-mini-2024-07-18"):
    messages = [{"role": "user", "content": prompt}]
    client = OpenAI(api_key=os.environ["OPENAI_KEY"])

    response = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=messages
    )

    return response.choices[0].message.content


In [5]:
# Test the function
get_completion("What is the capital of Japan?")

'The capital of Japan is Tokyo.'

## 2.- Calling the OpenAI API

Let's suppose that we own an organization that receives emails in multiple languages. We could use the OpenAI API to automatically translate these emails from any language to american english.

In [6]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [9]:
style = """American English \
in a calm and respectful tone"""

In [10]:
prompt = f"""Translate the text \
that is delimited by triple backsticks
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backsticks
into a style that is American English in a calm and respectful tone.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [12]:
response = get_completion(prompt)
print(response)

I am quite frustrated that the lid of my blender came off and splattered my kitchen walls with smoothie. To make matters worse, the warranty does not cover the cost of cleaning up my kitchen. I would appreciate your assistance with this issue. Thank you.


## 3.- Use the OpenAI API with LangChain

Now, let's perform this same process in a more convenient way using LangChain

In [15]:
# Load langchain abstraction for the ChatGPT endpoint
from langchain.chat_models import ChatOpenAI

In [21]:
chat = ChatOpenAI(model="gpt-4o-mini-2024-07-18", api_key=os.environ["OPENAI_KEY"], temperature=0.0)
# chat

In [25]:
# IMPORTANT: When defining a template string, we do not need to use a format string, LangChain ChatPromptTemplate automatically infers the parameters
template_string = """Translate the text \
that is delimited by triple backsticks \
into a style that is {style}. \
text: ```{text}```
"""

In [26]:
# Import this module to repeteadly use the template defined beforehand
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

In [27]:
# Print the prompt, we can observe that it includes 'text' and 'style' as parameters
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, template='Translate the text that is delimited by triple backsticks into a style that is {style}. text: ```{text}```\n')

In [29]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [30]:
customer_style = """American English \
in a calm and respectful tone"""

In [31]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [32]:
customer_messages = prompt_template.format_messages(style=customer_style, text=customer_email)

In [33]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [35]:
print(customer_messages[0])

content="Translate the text that is delimited by triple backsticks into a style that is American English in a calm and respectful tone. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n" additional_kwargs={} response_metadata={}


In [37]:
# customer_response = chat(customer_messages) DEPRECATED
customer_response = chat.invoke(customer_messages)

In [38]:
print(customer_response.content)

I’m quite frustrated that the lid of my blender came off and splattered smoothie all over my kitchen walls. To make matters worse, the warranty doesn’t cover the cost of cleaning up my kitchen. I could really use your help right now.


Now, let's translate the service response to the client language.

In [39]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [40]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [41]:
service_message = prompt_template.format_messages(style=service_style_pirate, text=service_reply)

In [42]:
service_message

[HumanMessage(content="Translate the text that is delimited by triple backsticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!\n```\n", additional_kwargs={}, response_metadata={})]

In [43]:
service_response = chat.invoke(service_message)

In [44]:
service_response

AIMessage(content="Ahoy there, esteemed customer! I must inform ye that the warranty be not coverin' the expenses for cleanin' yer galley, as it appears ye have misused yer blender by neglectin' to secure the lid afore settin' it to whirl. Aye, 'tis a bit of tough luck, I fear! Fair winds to ye, and may we cross paths again!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 80, 'prompt_tokens': 79, 'total_tokens': 159, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'stop', 'logprobs': None}, id='run--06c9b35e-0883-475b-82e2-6dade6e7e7f5-0')

## 3.- Output parser
LangChain is able to parse the output of an LLM into different formats.

Let's suppose that we want to process the response of an LLM an convert it into a JSON with specific keys.

In [45]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [46]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [48]:
prompt_template = ChatPromptTemplate.from_template(review_template)

In [49]:
print(prompt_template)

input_variables=['text'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'), additional_kwargs={})]


In [50]:
messages = prompt_template.format_messages(text=customer_review)

In [53]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.




In [54]:
response = chat.invoke(messages)

In [55]:
print(response.content)

```json
{
  "gift": true,
  "delivery_days": 2,
  "price_value": ["It's slightly more expensive than the other leaf blowers out there", "I think it's worth it for the extra features"]
}
```


In [56]:
# If we check the type of the response, it is a string
type(response.content)

str

Let's use LangChain paraser to extract the JSON

In [58]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

In [89]:
# Define one response schema per property
gift_schema = ResponseSchema(name="gift", type="boolean", description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days", type="integer",description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value", type="array of strings", description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, delivery_days_schema, price_value_schema]

In [90]:
# Create output parser
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [91]:
output_parser

StructuredOutputParser(response_schemas=[ResponseSchema(name='gift', description='Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.', type='boolean'), ResponseSchema(name='delivery_days', description='How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.', type='integer'), ResponseSchema(name='price_value', description='Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.', type='array of strings')])

In [92]:
format_instructions = output_parser.get_format_instructions()

In [93]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": boolean  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": integer  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": array of strings  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [94]:
# New review template
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

In [95]:
prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, format_instructions=format_instructions)

In [96]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```

In [97]:
response = chat(messages)

In [98]:
print(response.content)

```json
{
	"gift": true,
	"delivery_days": 2,
	"price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```


In [99]:
# Convert to dictionary
output_dict = output_parser.parse(response.content)

In [100]:
print(output_dict)

{'gift': True, 'delivery_days': 2, 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}


In [102]:
type(output_dict)

dict

In [101]:
output_dict["gift"]

True